<h1 style='color:blue'>DATA ANALYTICS PROJECT</h1>
<h2 style='color:maroon'>Global Terrorist</h2>
<h2 style='color:gold'>Analyst: Brian Perez</h2>

<h2> Importing libraries

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import folium
from folium.plugins import HeatMap

import warnings
warnings.filterwarnings('ignore')

In [12]:
global_terror = pd.read_csv('Global_terrorist.csv', encoding='ISO-8859-1')

In [13]:
global_terror

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,NaN,0,NaN,182,Somalia,11,...,NaN,"""Somalia: Al-Shabaab Militants Attack Army Che...","""Highlights: Somalia Daily Media Highlights 2 ...","""Highlights: Somalia Daily Media Highlights 1 ...",START Primary Collection,0,0,0,0,NaN
181687,201712310029,2017,12,31,NaN,0,NaN,200,Syria,10,...,NaN,"""Putin's 'victory' in Syria has turned into a ...","""Two Russian soldiers killed at Hmeymim base i...","""Two Russian servicemen killed in Syria mortar...",START Primary Collection,-9,-9,1,1,NaN
181688,201712310030,2017,12,31,NaN,0,NaN,160,Philippines,5,...,NaN,"""Maguindanao clashes trap tribe members,"" Phil...",NaN,NaN,START Primary Collection,0,0,0,0,NaN
181689,201712310031,2017,12,31,NaN,0,NaN,92,India,6,...,NaN,"""Trader escapes grenade attack in Imphal,"" Bus...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN


<h1>Regional Analysis</h1>

In [15]:
filtered_region = global_terror[
    (global_terror['region_txt'] == 'South Asia') &
    (global_terror['iyear'] >= 2010) &
    (global_terror['attacktype1_txt'] == 'Armed Assault') &
    (global_terror['nkill'] > 5)
]

aggregated_region = filtered_region.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')

aggregated_region['normalized_weight'] = (
    (aggregated_region['incident_count'] - aggregated_region['incident_count'].min()) /
    (aggregated_region['incident_count'].max() - aggregated_region['incident_count'].min())
)

heat_data = aggregated_region[['latitude', 'longitude', 'normalized_weight']].values.tolist()

m = folium.Map(location=[25, 80], zoom_start=5)

HeatMap(heat_data).add_to(m)

m.save('south_asia_assault_heatmap.html')
m


<h1>Crime Type Analysis</h1>

In [17]:
global_terror = global_terror.dropna(subset=["latitude", "longitude"])

crime_df = global_terror[global_terror["attacktype1_txt"] == "Armed Assault"]

aggregated_crime = crime_df.groupby(["latitude", "longitude"]).size().reset_index(name="incident_count")

aggregated_crime["normalized_weight"] = (
    (aggregated_crime["incident_count"] - aggregated_crime["incident_count"].min()) /
    (aggregated_crime["incident_count"].max() - aggregated_crime["incident_count"].min())
)

heat_data = aggregated_crime[["latitude", "longitude", "normalized_weight"]].values.tolist()

crime_map = folium.Map(location=[20, 0], zoom_start=2)

HeatMap(heat_data).add_to(crime_map)

crime_map.save("crime_type_heatmap.html")
crime_map


<h1>Casualty Analysis</h1>

In [19]:
# Load only necessary columns
use_cols = ['latitude', 'longitude', 'nkill', 'nwound']
global_terror = pd.read_csv('Global_terrorist.csv', encoding='ISO-8859-1', usecols=use_cols)

# Drop missing location data early
global_terror.dropna(subset=['latitude', 'longitude'], inplace=True)

# Fill the emplty value in casualty columns and convert to float
global_terror['nkill'] = pd.to_numeric(global_terror['nkill'], errors='coerce').fillna(0).astype('float32')
global_terror['nwound'] = pd.to_numeric(global_terror['nwound'], errors='coerce').fillna(0).astype('float32')

# Compute total casualties
global_terror['total_casualties'] = global_terror['nkill'] + global_terror['nwound']

# Filter for rows with any casualties
casualty_df = global_terror[global_terror['total_casualties'] > 0]

# Optional: round lat/lon to reduce unique locations (reduces grouping size)
casualty_df['latitude'] = casualty_df['latitude'].round(2)
casualty_df['longitude'] = casualty_df['longitude'].round(2)

# Group by rounded coordinates
aggregated_casualty = casualty_df.groupby(['latitude', 'longitude'])['total_casualties'].sum().reset_index()

min_c = aggregated_casualty['total_casualties'].min()
max_c = aggregated_casualty['total_casualties'].max()
aggregated_casualty['normalized_weight'] = (aggregated_casualty['total_casualties'] - min_c) / (max_c - min_c)

# Prepare data
heat_data = aggregated_casualty[['latitude', 'longitude', 'normalized_weight']].values.tolist()

# Create map
casualty_map = folium.Map(location=[20, 0], zoom_start=2)
HeatMap(heat_data).add_to(casualty_map)
casualty_map.save("casualty_heatmap.html")
casualty_map


<h1>Demographic Analysis</h1>

In [20]:
# removing the rows with missing coordinates
df.dropna(subset=['latitude', 'longitude'], inplace=True)

demographic_df = df[(df['gname'] == 'Black Nationalists') & (df['natlty1_txt'] == 'United States')]

aggregated_demographic = demographic_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')

# Normalize incident counts
min_inc = aggregated_demographic['incident_count'].min()
max_inc = aggregated_demographic['incident_count'].max()
aggregated_demographic['normalized_weight'] = (
    (aggregated_demographic['incident_count'] - min_inc) / (max_inc - min_inc)
)

heat_data = aggregated_demographic[['latitude', 'longitude', 'normalized_weight']].values.tolist()

demographic_insight = folium.Map(location=[37.0, -95.0], zoom_start=4)  # Focus on US
HeatMap(heat_data).add_to(demographic_insight)

demographic_insight.save('demographic_heatmap.html')
demographic_insight


<h1>Target Analysis</h1>

In [10]:
target_df = df[(df['targtype1_txt'] >= 'Government (General)') & (df['attacktype1_txt'] == 'Bombing/Explosion')]

aggregated_target = target_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')

aggregated_target['normalized_weight'] = (aggregated_target['incident_count'] - aggregated_target['incident_count'].min()) / \
                                     (aggregated_target['incident_count'].max() - aggregated_target['incident_count'].min())

heat_data = aggregated_target[['latitude', 'longitude', 'normalized_weight']].values.tolist()

target_insight = folium.Map(location = [30, 40], zoom_start = 4)

HeatMap(heat_data).add_to(target_insight)

target_insight.save('terrorism_heatmap.html')
target_insight